In [12]:
from torch import nn
import numpy as np
import torch
from torch.distributions.normal import Normal
import torch.optim as optim
import torch.utils.data

zeros = torch.zeros(1, 129, 8)
zeros = zeros.unsqueeze(0)
print(zeros.size())
zeroPadding = nn.ZeroPad2d((0, 0, 4, 4))
x = zeroPadding(zeros)
print(x.size())
conv = nn.Conv1d(1, 18, kernel_size=(9,8), padding='valid', bias=False)
x = conv(x)
print(x.size())
relu = nn.ReLU()
x = relu(x)
print(x.size())
bn = nn.BatchNorm2d(num_features=18)
x = bn(x)
print(x.size())

torch.Size([1, 1, 129, 8])
torch.Size([1, 1, 137, 8])
torch.Size([1, 18, 129, 1])
torch.Size([1, 18, 129, 1])
torch.Size([1, 18, 129, 1])


Preprocessing Data

In [3]:
from preprocess import startProcessing

voicefolder = "H:/Projects/2022-2023/ConvDenoiser/dataset/voice/"
noisefolder = "H:/Projects/2022-2023/ConvDenoiser/dataset/ambience/"
exportfolder = "H:/Projects/2022-2023/ConvDenoiser/dataset/output/"
savepathy = ""

startProcessing(savepathy, voicefolder, noisefolder, exportfolder, 'All')

Number of cpu's :  8


100%|██████████| 442/442 [37:06<00:00,  5.04s/it]  



Enjoy your freshly processed files


Make and test model

In [2]:
from SPDenoiser import SPDenoiser, SPDataset
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = torch.nn.MSELoss()
device="cpu"
input = torch.zeros(60, 8)
input = input.unsqueeze(2)
input = input.unsqueeze(0)
print(input.size())

spdenoiser = SPDenoiser(60, 8, device)
spdenoiser.to(device)
spdataset = SPDataset("H:/Projects/2022-2023/ConvDenoiser/dataset/output/sp/", "H:/Projects/2022-2023/ConvDenoiser/dataset/output/csp/")
spdataloader = DataLoader(spdataset, batch_size=64, shuffle=True)

optimizer=torch.optim.Adam(spdenoiser.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossy = 0
step = 0

for epoch in range(epochs):
    for steppy, (x_train,y_train) in tqdm(enumerate(spdataloader),desc=f"Training. Epoch: {epoch}. Loss for step {step}: {lossy}"):
        step = steppy
        x_train.double().to(device)
        y_train.double().to(device)

        # print(x_train.size())

        output = spdenoiser(x_train)
        # print("Output: ", output.size())
        # print("Y train: ", y_train.size())

        loss= loss_fn(output,y_train)
        lossy = loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    

input = torch.zeros(60, 8)
input = input.unsqueeze(2)
input = input.unsqueeze(0)
print(input.size())

Loading files to dataset:   7%|▋         | 32/442 [00:00<00:01, 311.09it/s]

torch.Size([1, 60, 8, 1])
Directories:  H:/Projects/2022-2023/ConvDenoiser/dataset/output/sp/ | H:/Projects/2022-2023/ConvDenoiser/dataset/output/csp/
Noisy file amount:  442
Clean file amount:  442


Loading files to dataset: 100%|██████████| 442/442 [00:01<00:00, 233.94it/s]
Training. Epoch: 0. Loss for step 0: 0: 0it [00:00, ?it/s]C:\Users\woute\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\modules\conv.py:453: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:883.)
  return F.conv2d(input, weight, bias, self.stride,
Training. Epoch: 0. Loss for step 0: 0: 382it [00:33,  9.53it/s]

KeyboardInterrupt: 

In [2]:
import torch
input = torch.zeros(60, 8)
print(input[-1:].size())

torch.Size([1, 8])


c:\Users\woute\.conda\envs\Diffwave\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
